<a href="https://colab.research.google.com/github/piabirajdar/AGENT_VERSION_CONTROL_CODE_REVIEW/blob/main/Agent_Version_Control_Code_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain_core langchain_community langgraph langchain_openai

In [ ]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('openAiKey')
os.environ['GITHUB_TOKEN'] = userdata.get('githubToken')

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [ ]:
from typing_extensions import TypedDict

class State(TypedDict):
  pr_diff: str
  change_category: str
  risk_level: str
  response: str


In [ ]:
from langgraph.graph import StateGraph, START, END

workflow = StateGraph(State)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
def categorize_change(state: State) -> State:
  print("\nCategorizing the code change : ")

  prompt = ChatPromptTemplate.from_template(
      """
      You are a senior engineer.

      Categorize this pull request as one of:
      Bugfix, Feature, Refactor, Config, Test.

      PR Title: {title}
      PR Description: {description}
      Files Changed: {files}
      Diff:
      {diff}
      """
  )

  chain = prompt | llm
  change_category = chain.invoke({
      "title": state["pr_title"],
      "description": state["pr_description"],
      "files": ", ".join(state["files_changed"]),
      "diff": state["pr_diff"]
  }).content.strip()

  print(f"Change Category : {change_category}")
  return {"change_category": change_category}


In [ ]:
def assess_risk(state: State) -> State:
  print("\nAssessing risk level : ")

  prompt = ChatPromptTemplate.from_template(
      """
      You are reviewing a pull request.

      Assess the risk as Low, Medium, or High.

      Consider:
      - Change category
      - CI status
      - Files touched
      - Size and impact of diff

      Change Category: {category}
      CI Status: {ci}
      Files Changed: {files}
      Diff:
      {diff}

      Respond with ONLY: Low, Medium, or High.
      """
  )

  chain = prompt | llm
  risk_level = chain.invoke({
      "category": state["change_category"],
      "ci": state["ci_status"],
      "files": ", ".join(state["files_changed"]),
      "diff": state["pr_diff"]
  }).content.strip()

  print(f"Risk Level : {risk_level}")
  return {"risk_level": risk_level}


In [ ]:
def auto_approve(state: State) -> State:
  print("\nAuto-approving PR : ")

  # normalize
  category = (state.get("change_category") or "").strip()
  risk = (state.get("risk_level") or "").strip()
  diff = state.get("pr_diff") or ""

  # lightweight “actual logic”: only auto-approve if it looks small/safe
  # (You can tighten/relax these heuristics.)
  looks_large = len(diff.splitlines()) > 200
  touches_sensitive = any(k in diff.lower() for k in [
      "payment", "auth", "encryption", "jwt", "token", "password", "schema", "migration", "terraform", "kubernetes"
  ])

  if looks_large or touches_sensitive or risk.lower() == "high":
    # fail-safe: don’t auto-approve if anything feels off
    return {
      "response": "Not auto-approved: change appears large or sensitive. Routing to fixes/escalation based on risk.",
      "email_subject": "PR Review: Auto-approval skipped (needs human attention)",
      "email_to": "team-review@company.com",
      "email_body": (
        f"Auto-approval was skipped.\n\n"
        f"Category: {category}\nRisk: {risk}\n"
        f"Reason: {'large diff' if looks_large else ''}{' + ' if looks_large and touches_sensitive else ''}"
        f"{'sensitive area touched' if touches_sensitive else ''}\n"
      )
    }

  return {
    "response": "PR has been auto-approved. Low-risk change.",
    "email_subject": f"PR Approved ✅ ({category or 'Uncategorized'})",
    "email_to": "pr-author@company.com",
    "email_body": (
      f"Approved ✅\n\n"
      f"Summary:\n"
      f"- Category: {category or 'N/A'}\n"
      f"- Risk: {risk or 'N/A'}\n\n"
      f"Notes:\n"
      f"- Thanks! This change looks safe and aligned with existing patterns.\n"
      f"- If this touches production behavior unexpectedly, please add a quick rollback note in the PR description.\n"
    )
  }


In [ ]:
def request_fixes(state: State) -> State:
  print("\nRequesting fixes : ")

  diff = state.get("pr_diff") or ""
  category = (state.get("change_category") or "").strip()
  risk = (state.get("risk_level") or "").strip()

  # Simple heuristics to produce “actual” feedback without needing extra tools.
  # You can expand these into richer checks later.
  missing_tests = ("test" not in diff.lower()) and ("spec" not in diff.lower())
  risky_patterns = []
  if "TODO" in diff:
    risky_patterns.append("Contains TODO markers")
  if "print(" in diff.lower() or "console.log" in diff.lower():
    risky_patterns.append("Debug logging left in code")
  if "except:" in diff.lower():
    risky_patterns.append("Bare exception handling detected")
  if "password" in diff.lower() or "token" in diff.lower():
    risky_patterns.append("Potential secret/security-sensitive handling")

  bullets = []
  if missing_tests:
    bullets.append("- Add/extend tests for the new/changed behavior (unit or integration).")
  if risky_patterns:
    bullets.append("- Address review issues:\n  - " + "\n  - ".join(risky_patterns))
  if not bullets:
    bullets.append("- Please add a brief explanation of the change rationale and expected impact in the PR description.")

  return {
    "response": "PR requires fixes before approval.",
    "email_subject": f"PR Changes Requested 🛠️ ({category or 'Uncategorized'})",
    "email_to": "pr-author@company.com",
    "email_body": (
      f"Changes requested 🛠️\n\n"
      f"Category: {category or 'N/A'}\n"
      f"Risk: {risk or 'N/A'}\n\n"
      f"Requested updates:\n"
      + "\n".join(bullets)
      + "\n\nOnce updated, re-request review and I’ll take another look."
    )
  }


In [ ]:
def escalate_to_senior_engineer(state: State) -> State:
  print("\nEscalating to senior engineer : ")

  diff = state.get("pr_diff") or ""
  category = (state.get("change_category") or "").strip()
  risk = (state.get("risk_level") or "").strip()

  # Identify common escalation triggers
  triggers = []
  lower = diff.lower()
  if any(k in lower for k in ["payment", "billing", "refund", "invoice"]):
    triggers.append("Billing/Payment logic")
  if any(k in lower for k in ["auth", "oauth", "jwt", "token", "password", "encryption"]):
    triggers.append("Security/Auth")
  if any(k in lower for k in ["schema", "migration", "alter table", "index"]):
    triggers.append("DB schema/migration")
  if any(k in lower for k in ["terraform", "kubernetes", "helm", "docker", "iam", "policy"]):
    triggers.append("Infra/Deployment")
  if len(diff.splitlines()) > 300:
    triggers.append("Large change set")

  trigger_text = ", ".join(triggers) if triggers else "High-risk/unclear impact"

  # Keep the email short but useful (senior engineers want signal, not noise)
  return {
    "response": "PR escalated due to high risk or critical changes.",
    "email_subject": f"Escalation: High-risk PR Review Needed ⚠️ ({category or 'Uncategorized'})",
    "email_to": "senior-engineer@company.com",
    "email_cc": "team-lead@company.com",
    "email_body": (
      f"Requesting senior review ⚠️\n\n"
      f"Category: {category or 'N/A'}\n"
      f"Risk: {risk or 'N/A'}\n"
      f"Why escalating: {trigger_text}\n\n"
      f"What I need from you:\n"
      f"- Validate correctness + edge cases\n"
      f"- Confirm rollout/rollback considerations\n"
      f"- Flag any security/data integrity concerns\n\n"
      f"PR context (short):\n"
      f"- Diff lines: {len(diff.splitlines())}\n"
      f"- Key areas touched: {trigger_text}\n"
    )
  }


In [ ]:
workflow.add_node("categorize_change", categorize_change)
workflow.add_node("assess_risk", assess_risk)
workflow.add_node("auto_approve", auto_approve)
workflow.add_node("request_fixes", request_fixes)
workflow.add_node("escalate_to_senior_engineer", escalate_to_senior_engineer)


In [ ]:
def route_pr(state: State) -> str:
  if state["risk_level"] == "Low":
    return "auto_approve"
  elif state["risk_level"] == "Medium":
    return "request_fixes"
  else:
    return "escalate_to_senior_engineer"


In [ ]:
workflow.add_edge(START, "categorize_change")
workflow.add_edge("categorize_change", "assess_risk")
workflow.add_conditional_edges("assess_risk", route_pr)
workflow.add_edge("auto_approve", END)
workflow.add_edge("request_fixes", END)
workflow.add_edge("escalate_to_senior_engineer", END)


In [ ]:
app = workflow.compile()

In [ ]:
from IPython.display import Image, display

display(
    Image(
        app.get_graph().draw_mermaid_png()
    )
)

In [ ]:
def run_code_review(
    pr_diff: str,
    pr_title: str,
    pr_description: str,
    files_changed: list,
    ci_status: str
):
  results = app.invoke({
      "pr_diff": pr_diff,
      "pr_title": pr_title,
      "pr_description": pr_description,
      "files_changed": files_changed,
      "ci_status": ci_status
  })

  return {
      "change_category": results["change_category"],
      "risk_level": results["risk_level"],
      "response": results["response"]
  }


In [ ]:
import requests

def fetch_pr_diff(owner, repo, pr_number):
   token = os.environ.get("GITHUB_TOKEN")
    if not token:
        raise RuntimeError("GITHUB_TOKEN is not set")

    url = f"https://api.github.com/repos/{owner}/{repo}/pulls/{pr_number}"
    headers = {
        "Accept": "application/vnd.github.v3.diff",
        "Authorization": f"Bearer {token}"
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.text
real_diff_from_github = fetch_pr_diff("piabirajdar", "Priyanka-LeetCode-Solutions", "2")
results = run_code_review(
    pr_diff=real_diff_from_github,
    pr_title="Fix NPE in UserService",
    pr_description="Handles null userId edge case and adds test",
    files_changed=["UserService.java", "UserServiceTest.java"],
    ci_status="passed"
)